## Widget Viewer

In [18]:
from PIL import Image as PILImage
import piexif  
import json
import os
import ipywidgets as widgets
from IPython.display import display
import pprint

# Assuming 'sd_samplers' and 'IGNORED_INFO_KEYS' are defined elsewhere in your project
#  import them as needed

# Functions

In [19]:
def read_info_from_image(image: PILImage.Image) -> tuple[str | None, dict]:
    """Reads metadata from a PIL Image object.

    Args:
        image: A PIL Image object.

    Returns:
        A tuple containing:
            - geninfo: The value of the 'parameters' key from the image metadata, or None if not present.
            - items: A dictionary containing the remaining metadata from the image.
    """

    # Get the image metadata (info dictionary) or an empty dictionary if no metadata is present
    items = (image.info or {}).copy()

    # Extract the 'parameters' value from the metadata and remove it from the dictionary
    geninfo = items.pop('parameters', None)

    # Return the extracted 'parameters' value and the remaining metadata
    return geninfo, items

In [21]:
def parsePrompts(prompt):
    """Parses a prompt string into positive and negative components.

    Args:
        prompt: The prompt string to parse.

    Returns:
        A tuple containing:
            - positive: The positive part of the prompt (up to "Steps:").
            - negative: The negative part of the prompt (after "Negative prompt:"), or an empty string if not present.
    """

    positive = ""  # Initialize positive prompt
    negative = ""  # Initialize negative prompt

    # Find the index of "Steps:" in the prompt
    steps_index = prompt.find('Steps:')
    if steps_index != -1:  # If "Steps:" is found
        # Extract the positive prompt (up to "Steps:")
        positive = prompt[:steps_index].strip()

    # Find the index of "Negative prompt:" within the positive prompt
    negative_index = positive.find('Negative prompt:')
    if negative_index != -1:  # If "Negative prompt:" is found
        # Extract the negative prompt (after "Negative prompt:")
        negative = positive[negative_index + len('Negative prompt:'):].strip()
        # Remove the negative prompt from the positive prompt
        positive = positive[:negative_index].strip()

    return positive, negative  # Return the positive and negative prompts

#Simple Test:
print(parsePrompts("This is a positive prompt. Negative prompt: This is a negative prompt. Steps: This is a step."))

('This is a positive prompt.', 'This is a negative prompt.')


# Thumbnails with hover over text (Title)

In [ ]:
import os
import base64
from ipywidgets import Image, HBox, VBox, HTML
from IPython.display import display


image_dir = 'checked'
image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

image_files = image_files[:10]  # Limit number of images to display

image_widgets = []
for image_file in image_files:
    image_path = os.path.join(image_dir, image_file)
    with open(image_path, 'rb') as f:
        image = Image(value=f.read(), format='jpg')  # Adjust format if needed
        image.width = 450  # Set thumbnail size

        # Encode the image data to Base64
        image_data_base64 = base64.b64encode(image.value.tobytes()).decode('utf-8')

        # Create an HTML widget to wrap the image with a title attribute
        html_widget = HTML(value=f'<img src="data:image/jpeg;base64,{image_data_base64}" title="{image_file}" width="{image.width}">')

        image_widgets.append(html_widget)

# Arrange thumbnails in a grid 
cols = 4
grid = VBox([HBox(image_widgets[i:i+cols]) for i in range(0, len(image_widgets), cols)])
display(grid)

# Thumbnails with hover over text (Positive Prompt)

In [22]:
import os
import base64
from PIL import Image as PILImage 
from ipywidgets import Image, HBox, VBox, HTML
from IPython.display import display

# Assuming you have the `read_info_from_image` and `parsePrompts` functions defined

image_dir = 'checked'
image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

image_files = image_files[:10] 

image_widgets = []
for image_file in image_files:
    image_path = os.path.join(image_dir, image_file)
    
    # Open the image using PIL
    with PILImage.open(image_path) as pil_image:
        # Read metadata from the image
        geninfo, _ = read_info_from_image(pil_image) 
        
        # Parse the prompt from geninfo
        positive_prompt, _ = parsePrompts(geninfo or "") 

        # Convert PIL Image to ipywidgets Image
        with open(image_path, 'rb') as f:
            image = Image(value=f.read(), format='jpg') 
            image.width = 450 

        image_data_base64 = base64.b64encode(image.value.tobytes()).decode('utf-8')

        # Use the positive prompt as the title (hover text)
        html_widget = HTML(value=f'<img src="data:image/jpeg;base64,{image_data_base64}" title="{positive_prompt}" width="{image.width}">')

        image_widgets.append(html_widget)

cols = 4
grid = VBox([HBox(image_widgets[i:i+cols]) for i in range(0, len(image_widgets), cols)])
display(grid)

In [ ]:
import os
from ipywidgets import Image, HBox, VBox
from IPython.display import display


image_dir = 'checked'
image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

image_files = image_files[:10]  # Limit number of images to display

image_widgets = []
for image_file in image_files:
    image_path = os.path.join(image_dir, image_file)
    with open(image_path, 'rb') as f:
        image = Image(value=f.read(), format='jpg')  # Adjust format if needed
        image.width = 450  # Set thumbnail size
        image_widgets.append(image)
        
# Arrange thumbnails in a grid 
cols = 4
grid = VBox([HBox(image_widgets[i:i+cols]) for i in range(0, len(image_widgets), cols)])
display(grid)

In [11]:
def parsePrompts(prompt):
  positive = ""
  negative = ""

  # Extract everything up to 'Steps:' into positive
  steps_index = prompt.find('Steps:')
  if steps_index != -1:
    positive = prompt[:steps_index].strip()

  # Check if 'Negative prompt:' exists within positive
  negative_index = positive.find('Negative prompt:')
  if negative_index != -1:
    # Extract everything after 'Negative prompt:' into negative
    negative = positive[negative_index + len('Negative prompt:'):].strip()
    # Remove 'Negative prompt:' and its content from positive
    positive = positive[:negative_index].strip()

  return positive, negative

('This is a positive prompt.', 'This is a negative prompt.')


In [ ]:
def read_info_from_image(image: Image.Image) -> tuple[str | None, dict]:
    items = (image.info or {}).copy()
    geninfo = items.pop('parameters', None)

    positive_prompt = None
    if geninfo:
        start_index = geninfo.find("Positive prompt:") 
        if start_index != -1:
            start_index += len("Positive prompt:")  # Move past the label
            end_index_steps = geninfo.find("Steps:", start_index)
            end_index_negative = geninfo.find("Negative prompt:", start_index)

            if end_index_steps != -1 and (end_index_negative == -1 or end_index_steps < end_index_negative):
                end_index = end_index_steps
            elif end_index_negative != -1:
                end_index = end_index_negative
            else:
                end_index = len(geninfo)  # Go to the end if neither is found

            positive_prompt = geninfo[start_index:end_index].strip()

    return positive_prompt, items

In [ ]:
def read_info_from_image(image: Image.Image) -> tuple[str | None, dict]:
    items = (image.info or {}).copy()
    geninfo = items.pop('parameters', None)

    positive_prompt = None
    if geninfo:
        end_index_steps = geninfo.find("Steps:")

        if end_index_steps != -1:
            positive_prompt = geninfo[:end_index_steps].strip()

    return positive_prompt, items

In [ ]:
from PIL import Image
import os

def read_info_from_image(image: Image.Image) -> tuple[str | None, dict]:
    items = (image.info or {}).copy()
    geninfo = items.pop('parameters', None)
    return geninfo, items

checked_dir = 'checked'  # Replace with the actual path

image_files = [f for f in os.listdir(checked_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

image_info_array = []

for image_file in image_files:
    image_path = os.path.join(checked_dir, image_file)

    try:
        with Image.open(image_path) as img:
            geninfo, items = read_info_from_image(img)
            image_info_array.append({
                'filename': image_file,
                'general_info': geninfo
            })

    except IOError:
        print(f"Error: Unable to open image {image_path}")

pprint.pprint(image_info_array)

print(image_info_array[0]['general_info'])

In [ ]:
print(read_info_from_image(image_files[0]))

In [ ]:
expCopyPath = "C:\pydev\forge\stable-diffusion-webui-forge\outputs\\txt2img-images\\2024-08-30"
image_dir = expCopyPath.replace("\\", "/")

In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image

# Define directories
# image_dir = 'C:\\pydev\\webui_forge_cu121_torch21\\webui\\output\\txt2img-images\\2024-08-22'  # Replace with your actual directory
# C:\pydev\stable-diffusion-webui\outputs\txt2img-images\2024-08-25
# image_dir = 'C:\\pydev\\stable-diffusion-webui\\outputs\\txt2img-images\\2024-08-26-20240826T211045Z-001\\2024-08-26'  # Replace with your actual directory
#  2024-08-26-20240826T211045Z-001
# C:\pydev\stable-diffusion-webui\outputs\txt2img-images\2024-08-26-20240826T211045Z-001\2024-08-26
image_dir = 'C:\\pydev\\forge\\stable-diffusion-webui-forge\\outputs\\txt2img-images\\2024-08-30'
checked_dir = 'checked\\noLoras'
deleted_dir = 'deleted'


widthsetting = '700px'

# Create subdirectories if they don't exist
for dir_name in [checked_dir, deleted_dir]:
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

# Get list of images in the directory
image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
current_image_index = 0
imageCount = len(image_files)



# Create widgets
image_widget = widgets.Image(layout=widgets.Layout(width=widthsetting))  # Adjust width as needed
checked_button = widgets.Button(description='Checked')
deleted_button = widgets.Button(description='Deleted')

def read_info_from_image(image: Image.Image) -> tuple[str | None, dict]:
    items = (image.info or {}).copy()

    geninfo = items.pop('parameters', None)

    return geninfo, items

# Function to update the displayed image
def update_image():
    if 0 <= current_image_index < len(image_files):
        image_path = os.path.join(image_dir, image_files[current_image_index])
        image = Image.open(image_path)
        image_widget.value = image._repr_png_()  # Convert image to bytes for display
        geninfo, items = read_info_from_image(image)
    else:
        image_widget.value = b''  # Clear image if no more images

# Function to handle button clicks
def on_button_click(button):
    global current_image_index

    if 0 <= current_image_index < len(image_files):
        image_name = image_files[current_image_index]
        source_path = os.path.join(image_dir, image_name)

        if button == checked_button:
            target_dir = checked_dir
        else:  # button == deleted_button
            target_dir = deleted_dir

        target_path = os.path.join(target_dir, image_name)
        os.rename(source_path, target_path)

        del image_files[current_image_index]  # Remove from list
        if current_image_index >= len(image_files):
            current_image_index -= 1  # Move back if at the end

        update_image()
        update_text_block()

# Attach button click handlers
checked_button.on_click(on_button_click)
deleted_button.on_click(on_button_click)

# Create the HTML text block widget
text_block = widgets.HTML(
    value="""
    <div style="width: 400px; padding: 10px; border: 1px solid #ccc; margin-right: 10px;">
        <h3>Image Details</h3>
        <p>Filename: <span id="filename"></span></p>
        <p>Dimensions: <span id="dimensions"></span></p>
        <p>Remaining Files: {len(image_files)}</p>
    </div>
    """,
    layout=widgets.Layout(height='auto')  # Adjust height as needed
)

# Function to update the text block content
def update_text_block():
    if 0 <= current_image_index < len(image_files):
        image_path = os.path.join(image_dir, image_files[current_image_index])
        image = Image.open(image_path)
        geninfo, items = read_info_from_image(image)

        # Escape < and > in the geninfo string and add color styling
        escaped_geninfo = str(geninfo).replace("<", "&lt;").replace(">", "&gt;")


        negative_prompt_start = escaped_geninfo.find('Negative prompt:')
        if negative_prompt_start != -1:
            escaped_geninfo = (
                escaped_geninfo[:negative_prompt_start] +
                '<span style="color: red;">' + 
                escaped_geninfo[negative_prompt_start:] +
                '</span>'
            )
        
        # Color everything after "Steps:" in blue
        steps_start = escaped_geninfo.find('Steps:')
        if steps_start != -1:
            escaped_geninfo = (
                escaped_geninfo[:steps_start + len('Steps:')] +  # Include "Steps:" in the original color
                '<span style="color: blue;">' + 
                escaped_geninfo[steps_start + len('Steps:') :] +
                '</span>'
            )

        # Alternate colors before "Negative prompt:"
        # negative_prompt_start = escaped_geninfo.find('Negative prompt:')
        negative_prompt_start = escaped_geninfo.find('Steps:')
        if negative_prompt_start != -1:
            before_negative_prompt = escaped_geninfo[:negative_prompt_start]
            segments = before_negative_prompt.split(',')
            colored_segments = []
            for i, segment in enumerate(segments):
                color = "blue" if i % 2 == 0 else "green"
                colored_segments.append(f'<span style="color: {color};">{segment},</span>') 
            colored_before_negative_prompt = ''.join(colored_segments)
            escaped_geninfo = colored_before_negative_prompt + escaped_geninfo[negative_prompt_start:]

        text_block.value = f"""
        <div style="width: 400px; padding: 10px; border: 1px solid #ccc; margin-right: 10px;">
            <h3>Image Details</h3>
            <p>Filename: {image_files[current_image_index]}</p>
            <p>Dimensions: {image.width} x {image.height}</p>
            <p>Remaining Files: {len(image_files)}</p>
            <p>General Info: {escaped_geninfo}</p> 
        </div>
        """

    else:
        text_block.value = ""  # Clear text if no more images

# Update text block initially and when image changes
update_text_block()
update_image()  # Call after update_text_block to ensure both are updated

# Display widgets with the text block on the left
display(widgets.HBox([text_block, widgets.VBox([image_widget, widgets.HBox([checked_button, deleted_button])])]))





# # Initial display
# update_image()

# # Display widgets
# display(image_widget)
# display(widgets.HBox([checked_button, deleted_button]))